In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random

Using TensorFlow backend.


In [2]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [3]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [4]:
def DataSet(iFold):
    
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
        
    X_train = np.empty((nrow_train, side_len, side_len, 1))
    Y_train = np.empty((nrow_train, num_class))

    for index in range(nrow_train):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    X_train[index,i,j,0] = data_train[index,tmp_cnt]
                else:
                    X_train[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_train[index] = tmp_v
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1))
    Y_test  = np.empty((nrow_test, num_class))

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    X_test[index,i,j,0] = data_test[index,tmp_cnt]
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_train[index] = tmp_v
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    X_train_new = np.empty((nrow_train, 224, 224, 3))
    Y_train_new = Y_train
    
    for index in range(nrow_train):
        tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
        for i in range(224):
            for j in range(224):
                for c in range(3):
                    X_train_new[index,i,j,c] = tmp_new[i,j,0]
    
    X_test_new = np.empty((nrow_test, 224, 224, 3))
    Y_test_new = Y_test
    
    for index in range(nrow_test):
        tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
        for i in range(224):
            for j in range(224):
                for c in range(3):
                    X_test_new[index,i,j,c] = tmp_new[i,j,0]

    return X_train_new,Y_train,X_test_new,Y_test

In [5]:
models = []
for fold in range(10):
    iFold = fold + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    print(X_train[0])
    print(Y_train[0,1])

    # # model
    model = ResNet50(
        weights=None,
        classes=2
    )

    model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # # train
    model.fit(X_train, Y_train, epochs=1, batch_size=6)

    # # evaluate
    model.evaluate(X_test, Y_test, batch_size=32)
    
    # #
    models.append(model)

    # # save
    save_path =("./my_resnet_model.h5_F%d") % (iFold)
    model.save(save_path)

X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
[[[0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  ...
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]]

 [[0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  ...
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]]

 [[0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  [0.54220287 0.54220287 0.54220287]
  ...
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]
  [0.52764226 0.52764226 0.52764226]]

 ...

 [[0.31019921 0.31019921 0.31019921]
  [0.31019921 0.31019921 0.31019921]
  [0.31019921 0.31019921 0.31019921]
  ...
  [0.5        0.5        0.5       ]
  [0.5        0.5        0

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\keras\models.py:124: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
[[[0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  ...
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]]

 [[0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  ...
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]]

 [[0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  [0.50912194 0.50912194 0.50912194]
  ...
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]
  [0.41073429 0.41073429 0.41073429]]

 ...

 [[0.3183294  0.3183294  0.3183294 ]
  [0.3183294  0.3183294  0.3183294 ]
  [0.3183294  0.3183294  0.3183294 ]
  ...
  [0.5        0.5        0.5       ]
  [0.5        0.5        0

Epoch 1/1
157/157 [==============================] - 102s 648ms/step
X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
[[[0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  ...
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]]

 [[0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  ...
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]]

 [[0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  [0.34023258 0.34023258 0.34023258]
  ...
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]
  [0.51741938 0.51741938 0.51741938]]

 ...

 [[0.32180036 0.32180036 0.32180036]
  [0.32180036 0.32180036 0.32180036]
  [0.32180036 0.32180036 0.32180036]


MemoryError: 

In [6]:
X_train,Y_train,X_test,Y_test = DataSet(1)
true_labels = np.zeros(len(Y_test))
print(len(Y_test))

MemoryError: 

In [ ]:
all_precisions=[]
all_recalls=[]
all_Fvalues=[]
nRun = 10
num_class = 2
for iRun in range(nRun):
    iFold = iRun + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    print(X_train[0])
    print(Y_train[0,1])
    
    tmp_precision = np.zeros(num_class)
    tmp_recall    = np.zeros(num_class)
    tmp_Fvalue    = np.zeros(num_class)
    tmp_beta = 1
    model = models[iRun]
    yhat_label = model.predict_classes(X_test, batch_size=32, verbose=1)
    true_labels = np.zeros(len(Y_test))
    N_TP = np.zeros(num_class)
    N_FP = np.zeros(num_class)
    N_TN = np.zeros(num_class)
    N_FN = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = cur_test_label[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    N_TP[j]=N_TP[j]+1
                else:
                    N_FP[j]=N_FP[j]+1
            else:
                if j == true_label:
                    N_FN[j]=N_FN[j]+1
                else:
                    N_TN[j]=N_TN[j]+1
            
    for i in range(num_class):
        if N_TP[i] > 0:
            tmp_precision[i] = N_TP[i] / (N_TP[i] + N_FP[i])
            tmp_recall[i]    = N_TP[i] / (N_TP[i] + N_FN[i])
            tmp_Fvalue[i]=(1 + tmp_beta*tmp_beta)*tmp_recall[i]*tmp_precision[i]/\\
                          (tmp_beta*tmp_beta*(tmp_recall[i]+tmp_precision[i]))
        else:
            tmp_precision[i] = 0
            tmp_recall[i] = 0
            tmp_Fvalue[i] = 0
    
    all_precisions.append(tmp_precision)
    all_recalls.append(tmp_recall)
    all_Fvalues.append(tmp_Fvalue)


In [ ]:
# # restore
model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

img = image.load_img(img_path, target_size=(224, 224))

plt.imshow(img)
img = image.img_to_array(img)/ 255.0
img = np.expand_dims(img, axis=0)  # 为batch添加第四维

print(model.predict(img))
np.argmax(model.predict(img))